# Notebook to make Fig. 4

In [1]:
# Import necessary packages & set-up plotting aesthetics

import numpy as np 
import pylab
import pandas as pd
from scipy.stats import gaussian_kde

import json
import random

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from matplotlib import style

from helper_functions import *

style.use('plotting.mplstyle')

In [2]:
# TODO: create drawChiEff script
sys.path.append('/home/zoe.ko/LIGOSURF22/scripts/')
from drawChiEff import *

### Load data

In [9]:
# location of emcee samples
data_folder = '/home/simona.miller/comp-spin-mock-injections/Data/PopulationInferenceOutput/betaPlusGaussian/'
model = 'betaPlusGaussian'
numinjections = 70

all_data = []
for pop in np.array([1, 2, 3]):
    pop = str(pop)
    
    name = f'{data_folder}betaPlusGaussian_pop{pop}_{numinjections}events_temp'
    with open(f'{name}.json', 'r') as f:
        pop_data = json.load(f)
        all_data += [pop_data]

In [10]:
# location of reweighted sample dicts (output of running posterior_reweighting)
rw_folder = '/home/zoe.ko/comp-spin-mock-injections/Data/PopulationInferenceOutput/betaPlusGaussian/'

### Do posteriors reweighting and make figure

In [4]:
labels_dict = {
    'pop1':r'$\mathrm{\textsc{HighSpinPrecessing}}$', 
    'pop2':r'$\mathrm{\textsc{MediumSpin}}$', 
    'pop3':r'$\mathrm{\textsc{LowSpinAligned}}$'
}

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(16,10))

for runs in np.arange(100):
    all_chi1_pred = []
    all_chi2_pred = []
    all_cost1_pred = []
    all_cost2_pred = []
    all_chieff_pred = []

    all_chi1_obs = []
    all_chi2_obs = []
    all_cost1_obs = []
    all_cost2_obs = []
    all_chieff_obs = []

    for i,key in enumerate(['pop1', 'pop2', 'pop3']):
        
        # spin magnitudes
        axes[i][0].plot([0, 1], [0, 1], color='k') # diagonal line
        axes[i][0].set_xlim(0,1)
        axes[i][0].set_ylim(0,1)

        axes[i][1].plot([0, 1], [0, 1], color='k')
        axes[i][1].set_xlim(0,1)
        axes[i][1].set_ylim(0,1)

        # tilt angles
        axes[i][2].plot([-1, 1], [-1, 1], color='k')
        axes[i][2].set_xlim(-1,1)
        axes[i][2].set_ylim(-1,1)

        axes[i][3].plot([-1, 1], [-1, 1], color='k')
        axes[i][3].set_xlim(-1,1)
        axes[i][3].set_ylim(-1,1)

        # set label
        axes[i][0].set_ylabel(f'{labels_dict[key]}\n\nPredicted', fontsize=15)

        if i!=2: 
            for ax in axes[i]: 
                ax.set_xticklabels([])
        else: 
            for ax in axes[i]: 
                ax.set_xlabel('Observed', fontsize=15)

        int_pop = i
        pop = str(int_pop+1)
        data = all_data[int_pop]

        chi1_pred = []
        chi2_pred = []
        cost1_pred = []
        cost2_pred = []
        chieff_pred = []

        # Define a grid of effective spins
        chi_grid = np.linspace(0,1,1000)
        cost_grid = np.linspace(-1,1,1000)

        # Choose a number of traces to draw
        nTraces = int(numinjections)
        n = 1

        # Instantiate an array to hold a collection of traces
        p_chi = np.zeros((nTraces,chi_grid.size))
        p_cost = np.zeros((nTraces,cost_grid.size))
        chieff_kdes_on_grid = np.zeros((nTraces,cost_grid.size))

        # For a random set of posterior samples, compute and store the corresponding probability densities across our grid
        random_indices = np.random.choice(np.arange(len(data['mu_chi']['processed'])), size=nTraces,replace=False)
        for i,ind in enumerate(random_indices):

            mu_chi = data['mu_chi']['processed'][ind]
            sigma_chi = data['sigma_chi']['processed'][ind]
            mu_cost = data['mu_cost']['processed'][ind]
            sigma_cost = data['sigma_cost']['processed'][ind]
            Bq = data['Bq']['processed'][ind]

            # calculate Gaussian distribution for chi
            p_chi[i,:] = calculate_Gaussian_1D(chi_grid, mu_chi, sigma_chi, -1, 1)

            # and Gaussian distribution for cos(theta)
            p_cost[i,:] = calculate_Gaussian_1D(cost_grid, mu_cost, sigma_cost, -1, 1)

            # draw uniform component spins + masses
            nRandomDraws = 10000
            samp_idxs = np.arange(nRandomDraws)
            chi1s = np.random.rand(nRandomDraws)
            chi2s = np.random.rand(nRandomDraws)
            cost1s = np.random.rand(nRandomDraws)*2 - 1
            cost2s = np.random.rand(nRandomDraws)*2 - 1
            mAs = np.random.rand(nRandomDraws)*100
            mBs = np.random.rand(nRandomDraws)*100
            m1s = np.maximum(mAs, mBs)
            m2s = np.minimum(mAs, mBs)


            p_chi1 = calculate_Gaussian_1D(chi1s, mu_chi, sigma_chi, 0, 1)
            p_chi2 = calculate_Gaussian_1D(chi2s, mu_chi, sigma_chi, 0, 1)
            p_cost1 = calculate_Gaussian_1D(cost1s, mu_cost, sigma_cost, -1, 1)
            p_cost2 = calculate_Gaussian_1D(cost2s, mu_cost, sigma_cost, -1, 1)
            p_masses = p_astro_masses(m1s, m2s, bq=Bq)

            weights = p_chi1*p_chi2*p_cost1*p_cost2*p_masses
            weights_normed = weights/np.sum(weights)
            weights_normed[np.where(weights_normed<0)] = 0 # get rid of tiny division errors

            # select a subset of the samples subject to the weights
            # calculated from p(spins,masses)
            idxs = np.random.choice(samp_idxs, p=weights_normed, size=n)  

            # calculate chi-eff for these samples
            q = m2s[idxs]/m1s[idxs]
            chi_eff = (chi1s[idxs]*cost1s[idxs] + q*chi2s[idxs]*cost2s[idxs])/(1+q)

            chi1 = chi1s[idxs]
            chi2 = chi2s[idxs]
            cost1 = cost1s[idxs]
            cost2 = cost2s[idxs]

            chi1_pred = np.append(chi1_pred, chi1)
            chi2_pred = np.append(chi2_pred, chi2)
            cost1_pred = np.append(cost1_pred, cost1)
            cost2_pred = np.append(cost2_pred, cost2)
            chieff_pred = np.append(chieff_pred, chi_eff)



            # draw 500 chi-effective samples corresponding to the component spin distributions with hyper-parameters
            # {mu_chi, sigma_chi, MF_cost, sigma_cost, f_spike, sigma_spike, zmin, Bq,}
            chi_effs = drawChieffsGauss(mu_chi, sigma_chi, mu_cost, sigma_cost, -1, Bq, n=500)

            # calculate KDE from these draws and evaluate them on a grid of chi-effective values (i.e. chieff_grid = 
            # np.linspace(-1,1,100)
            chieff_grid = np.linspace(-1,1,1000)
            chieff_kde = gaussian_kde(chi_effs)
            chieff_kdes_on_grid[i,:] = chieff_kde(chieff_grid)



        all_chi1_pred += [chi1_pred]
        all_chi2_pred += [chi2_pred]
        all_cost1_pred += [cost1_pred]
        all_cost2_pred += [cost2_pred]
        all_chieff_pred += [chieff_pred]





        sampleDict_rw = np.load(rw_folder + str(numinjections) + model + '_' + 'pop' + pop
                                + 'rw_sampleDict.pickle', allow_pickle=True)
        
        chi1_obs = []
        chi2_obs = []
        cost1_obs = []
        cost2_obs = []
        chieff_obs = []

        for key in sampleDict_rw.keys():
            d = sampleDict_rw[key]
            chi1 = random.choice(d['chi1'])
            chi1_obs += [chi1]
            chi2 = random.choice(d['chi2'])
            chi2_obs += [chi2]
            cost1 = random.choice(d['cost1'])
            cost1_obs += [cost1]
            cost2 = random.choice(d['cost2'])
            cost2_obs += [cost2]

        all_chi1_obs += chi1_obs
        all_chi2_obs += chi2_obs
        all_cost1_obs += cost1_obs
        all_cost2_obs += cost2_obs
        all_chieff_obs += chieff_obs
    
        
        axes[int_pop][0].plot(np.sort(chi1_pred), np.sort(chi1_obs), alpha=0.3, c='cornflowerblue')
        axes[int_pop][0].plot(np.sort(chi1_pred), np.sort(chi1_pred), alpha = 0.3, ls='--', c='tomato')
        
        axes[int_pop][1].plot(np.sort(chi2_pred), np.sort(chi2_obs), alpha=0.3, c='cornflowerblue')
        axes[int_pop][1].plot(np.sort(chi2_pred), np.sort(chi2_pred), alpha = 0.3, ls='--', c='tomato')       

        axes[int_pop][2].plot(np.sort(cost1_pred), np.sort(cost1_obs), alpha=0.3, c='cornflowerblue')
        axes[int_pop][2].plot(np.sort(cost1_pred), np.sort(cost1_pred), alpha = 0.3, ls='--', c='tomato')
        
        axes[int_pop][3].plot(np.sort(cost2_pred), np.sort(cost2_obs), alpha=0.3, c='cornflowerblue')
        axes[int_pop][3].plot(np.sort(cost2_pred), np.sort(cost2_pred), alpha = 0.3, ls='--', c='tomato')

axes[0][0].set_title(r'$\chi_1$', fontsize=16, y=1.05)
axes[0][1].set_title(r'$\chi_2$', fontsize=16, y=1.05)
axes[0][2].set_title(r'$\cos\theta_1$', fontsize=16, y=1.05)
axes[0][3].set_title(r'$\cos\theta_2$', fontsize=16, y=1.05)

for ax in axes: 
    for a in ax:
        a.grid(ls=':', color='silver', lw=0.5)
        
plt.subplots_adjust(hspace=0.1)
    
plt.savefig('figure_04_temp.pdf',bbox_inches='tight')
plt.show()

/home/zoe.ko/comp-spin-mock-injections/Figures/./../Code/PopulationInference/posterior_helper_functions.py:227: RuntimeWarning: overflow encountered in exp
  f = np.exp(deltaM/m + deltaM/(m-deltaM))


### Make figure

In [ ]:
labels_dict = {
    'pop1':r'$\mathrm{\textsc{HighSpinPrecessing}}$', 
    'pop2':r'$\mathrm{\textsc{MediumSpin}}$', 
    'pop3':r'$\mathrm{\textsc{LowSpinAligned}}$'
}

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(16,10))

for i,key in enumerate(['pop1', 'pop2', 'pop3']):
    
    # spin magnitudes
    axes[i][0].plot([0, 1], [0, 1], color='k') # diagonal line
    axes[i][0].set_xlim(0,1)
    axes[i][0].set_ylim(0,1)
    
    axes[i][1].plot([0, 1], [0, 1], color='k')
    axes[i][1].set_xlim(0,1)
    axes[i][1].set_ylim(0,1)
    
    # tilt angles
    axes[i][2].plot([-1, 1], [-1, 1], color='k')
    axes[i][2].set_xlim(-1,1)
    axes[i][2].set_ylim(-1,1)
    
    axes[i][3].plot([-1, 1], [-1, 1], color='k')
    axes[i][3].set_xlim(-1,1)
    axes[i][3].set_ylim(-1,1)
    
    # set label
    axes[i][0].set_ylabel(f'{labels_dict[key]}\n\nPredicted', fontsize=15)
        
    if i!=2: 
        for ax in axes[i]: 
            ax.set_xticklabels([])
    else: 
        for ax in axes[i]: 
            ax.set_xlabel('Observed', fontsize=15)
    
axes[0][0].set_title(r'$\chi_1$', fontsize=16, y=1.05)
axes[0][1].set_title(r'$\chi_2$', fontsize=16, y=1.05)
axes[0][2].set_title(r'$\cos\theta_1$', fontsize=16, y=1.05)
axes[0][3].set_title(r'$\cos\theta_2$', fontsize=16, y=1.05)

for ax in axes: 
    for a in ax:
        a.grid(ls=':', color='silver', lw=0.5)
        
plt.subplots_adjust(hspace=0.1)
    
plt.savefig('figure_04_temp.pdf',bbox_inches='tight')
plt.show()